In [1]:
cd ..

/tmp/ml-cost-function


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

from sklearn.compose import * 
from sklearn.preprocessing import *
from sklearn.feature_extraction import * 
from sklearn.feature_selection import *

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as Pipeline

In [51]:
data = pd.read_csv('data/data.csv')
data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION
0,0,Cash loans,F,Y,Y,0,180000.0,755190.0,36459.0,675000.0,Family,Commercial associate,Secondary / secondary special,Married,House / apartment,0.008625,-18667,-2150,-716.0
1,0,Cash loans,M,N,Y,0,270000.0,868797.0,34587.0,702000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.046220,-17701,-1132,-4002.0
2,1,Cash loans,M,Y,Y,0,315000.0,1006920.0,39933.0,900000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.028663,-15597,-3752,-3722.0
3,1,Cash loans,M,Y,N,2,342000.0,476509.5,37647.0,360000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.002506,-13414,-1415,-631.0
4,0,Cash loans,F,Y,Y,1,171000.0,332473.5,22347.0,274500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.006008,-10710,-277,-4892.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0,Cash loans,F,N,N,1,202500.0,814041.0,23800.5,679500.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.022800,-18405,365243,-4734.0
304,0,Revolving loans,F,N,Y,0,90000.0,202500.0,10125.0,202500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Separated,House / apartment,0.005144,-18960,-428,-787.0
305,0,Revolving loans,F,N,N,0,72000.0,157500.0,7875.0,157500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.008068,-18097,-3970,-6309.0
306,0,Cash loans,M,Y,Y,0,225000.0,265500.0,20677.5,265500.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.018209,-16266,-972,-8959.0


In [52]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),

    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, make_column_selector(dtype_include=np.number)),
        ("cat", categorical_transformer, make_column_selector(dtype_exclude =np.number)),
    ]
)

In [53]:
X = data.drop(columns=['TARGET'])
y = data['TARGET']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [54]:
pipeline = Pipeline([
    ('sampler', RandomUnderSampler()),
    ("transformer", preprocessor),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('estimator', LogisticRegression()),
])

In [55]:
grid = GridSearchCV(pipeline, param_grid={}, cv=5, scoring='f1')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sampler', RandomUnderSampler()),
                                       ('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f5266ca1870>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f5266ca2f50>)])),
                                       ('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaler', StandardScaler()),
                                       ('estimator', LogisticRegression())]),
             param_grid={}, scoring='f1')

In [56]:
res  = pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score').head(5)

In [57]:
columns = [k for k in res.columns  if "split" not in k ]
res = res.loc[:, columns].round(2)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score
0,0.03,0.01,0.01,0.0,{},0.08,0.05,1


## First approch

In [73]:
def calcul_pnl_one_easy(montant, y_true, y_pred, taux=0.04, default_rate=0.25):

    if y_pred : # ==1
        return 0
    
    # y_pred == 0
    if y_true : # ==1
        return -default_rate * montant

    return montant * taux

In [74]:
calcul_pnl_one_easy(1000, 0, 0)

40.0

In [75]:
calcul_pnl_one_easy(1000, 0, 1)

0

In [76]:
calcul_pnl_one_easy(1000, 1, 0)

-250.0

In [77]:
calcul_pnl_one_easy(1000, 1, 1)

0

In [78]:
y_pred_list = grid.predict(X)
y_pred_list 

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,

In [79]:
y_true_list = y.values
y_true_list

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,

In [80]:
montant_list = X.loc[:, 'AMT_CREDIT'].values
montant_list

array([ 755190. ,  868797. , 1006920. ,  476509.5,  332473.5,  656811. ,
       1078200. ,  494550. ,  270000. ,  675000. ,  528633. , 1528200. ,
        609187.5,  337500. ,  450000. ,  675000. , 1046142. ,  675000. ,
        948582. , 1800000. ,  284400. ,  781920. , 1024290. ,  640080. ,
        441000. ,  585000. , 1024740. ,  521280. ,  473760. ,  199080. ,
        450000. ,  675000. ,  315000. ,  202500. ,  896157. ,  225000. ,
        453514.5,  675000. ,  460858.5,  630000. ,  286704. ,  364500. ,
        900000. ,  254700. ,  752116.5,  107820. ,  742500. ,  787500. ,
       1125171. ,  500566.5,  225000. ,  587619. ,  651595.5,  675000. ,
        675000. ,  291915. , 1006920. ,  479700. , 1350000. ,  337500. ,
       1072809. , 1193580. ,  247500. ,  900000. ,  269550. ,  675000. ,
        352044. ,   79128. ,  167895. ,  743031. ,  526500. ,  194076. ,
        180000. ,  545040. ,  645903. ,  247275. ,  270000. ,  728460. ,
        760225.5,  305221.5,  270000. ,  945000. , 

In [81]:
v = 0
for m, y_true, y_pred in zip(montant_list, y_true_list, y_pred_list):
    v += calcul_pnl_one_easy(m, y_true=y_true, y_pred=y_pred)
v

2573521.965

## 02 predict proba

In [83]:
y_pred_proba_list = grid.predict_proba(X)
y_pred_proba_list = list(y_pred_proba_list[ :, 0])
y_pred_proba_list = [round(x, 4) for x in y_pred_proba_list]
y_pred_proba_list

[0.6885,
 0.434,
 0.2597,
 0.2127,
 0.6593,
 0.5847,
 0.4907,
 0.0139,
 0.6513,
 0.766,
 0.6583,
 0.6148,
 0.3979,
 0.0358,
 0.0145,
 0.0402,
 0.0484,
 0.9068,
 0.765,
 0.04,
 0.7753,
 0.6405,
 0.3797,
 0.5104,
 0.3684,
 0.7573,
 0.8979,
 0.7163,
 0.526,
 0.7927,
 0.113,
 0.6748,
 0.3171,
 0.5643,
 0.667,
 0.226,
 0.8362,
 0.8796,
 0.6999,
 0.8304,
 0.533,
 0.5889,
 0.1569,
 0.473,
 0.1599,
 0.991,
 0.0141,
 0.9202,
 0.0769,
 0.6371,
 0.8604,
 0.024,
 0.2984,
 0.8019,
 0.8154,
 0.2599,
 0.8804,
 0.0432,
 0.7987,
 0.5453,
 0.3259,
 0.7401,
 0.6833,
 0.2127,
 0.2287,
 0.6307,
 0.2196,
 0.8457,
 0.6715,
 0.6804,
 0.3874,
 0.6482,
 0.4659,
 0.6823,
 0.6086,
 0.0657,
 0.7218,
 0.3792,
 0.7104,
 0.1786,
 0.5494,
 0.087,
 0.1293,
 0.5476,
 0.5348,
 0.6821,
 0.0677,
 0.6505,
 0.5189,
 0.3731,
 0.3484,
 0.185,
 0.8652,
 0.467,
 0.4998,
 0.1294,
 0.5771,
 0.4567,
 0.7468,
 0.8991,
 0.6432,
 0.499,
 0.5568,
 0.3613,
 0.057,
 0.8275,
 0.8327,
 0.1317,
 0.797,
 0.3993,
 0.1926,
 0.7384,
 0.0044,
 0

In [84]:
def calcul_pnl_one_complexe(montant, 
                            y_true, 
                            y_pred_proba, 
                            taux=0.04, 
                            default_rate=0.25, 
                            threshold=0.5, ):


    y_pred = 0 if y_pred_proba > threshold else 1

    if y_pred : 
        return 0
    
    if y_true :
        return -0.25 * montant

    return montant * taux

In [85]:
v = 0
for m, y_true, y_pred_proba in zip(montant_list, y_true_list, y_pred_proba_list):
    v += calcul_pnl_one_complexe(m, y_true=y_true, y_pred_proba=y_pred_proba)
v

2573521.965

In [87]:
v = 0
for m, y_true, y_pred_proba in zip(montant_list, y_true_list, y_pred_proba_list):
    v += calcul_pnl_one_complexe(m, y_true=y_true, y_pred_proba=y_pred_proba,threshold=0.0)
v

2959661.970000001

In [88]:
v = 0
for m, y_true, y_pred_proba in zip(montant_list, y_true_list, y_pred_proba_list):
    v += calcul_pnl_one_complexe(m, y_true=y_true, y_pred_proba=y_pred_proba,threshold=0.999)
v

0